Я выбрал VAE Autoencoder, так как он с легкостью позволяет генерировать новые изображения, всего лишь передав ему одну единственную точку (x,y).

В качестве датасета был использован следующий датасет https://www.kaggle.com/sshikamaru/fruit-recognition. Я выбрал датасет, содержащий изображения небольшого разрешения, всего 100х100 пикселей, так как VAE encoder  плохо подходит для генерации изображений высокого качества, лучше использовать GAN модели

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras import backend as K
import os
from collections import defaultdict
from shutil import copy
from shutil import copytree, rmtree
import PIL
import glob
from keras.preprocessing.image import img_to_array, array_to_img
from keras import layers
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
from keras.losses import binary_crossentropy
import imageio
!pip install -q git+https://github.com/tensorflow/docs
import tensorflow_docs.vis.embed as embed

Без этой настройки, модель отказывается обучаться (

In [ ]:
 tf.compat.v1.disable_eager_execution()

Я решил создать мини-выборку из 4 классов, именно эти классы выбрал для большей наглядности (у каждого класса либо цвет, либо форма другая)

In [ ]:
food_list = ['Apple Granny Smith','Corn','Onion White','Limes']
src_train = '/kaggle/input/fruit-recognition/train/train'
dest_train = 'train_mini'



In [ ]:
# Вспомогательная функция для создания мини-датасетов
def dataset_mini(food_list, src, dest):
  if os.path.exists(dest):
    rmtree(dest) # удаляем папку мини-датасета (если такая есть), для того, чтобы в папке были только те классы, которые мы укажем
  os.makedirs(dest)
  for food_item in food_list :
    print("Copying images into",food_item)
    copytree(os.path.join(src,food_item), os.path.join(dest,food_item))
      

In [ ]:
print("Создание тренировочной выборки с выбранными классами")
dataset_mini(food_list, src_train, dest_train)

In [ ]:
print("Всего изображений в мини тренировочной выборке")

!find train_mini -type d -or -type f -printf '.' | wc -c

In [ ]:
img_width, img_height = 112, 112
IMG_SHAPE = (img_width, img_height, 3)
latent_dim = 2

In [ ]:
datagen = ImageDataGenerator(rescale=1./ 255)

In [ ]:
train_gen = datagen.flow_from_directory('train_mini/',  class_mode='input', 
                                        target_size=(img_width, img_height), batch_size=32, color_mode = 'rgb')

Ниже описаны функции, необходимые для работы VAE, в частности своя loss функция для модели - совокупность Reconstruction-loss и KL-loss. А compute_latent это некий Reparameterization trick, позволяющий использовать градиентный спуск для нашей loss функции.

In [ ]:
def compute_latent(x):
    mu, sigma = x
    batch = K.shape(mu)[0]
    dim = K.int_shape(mu)[1]
    eps = K.random_normal(shape=(batch,dim))
    return mu + K.exp(sigma/2)*eps

In [ ]:
def kl_reconstruction_loss(true, pred):
    # Reconstruction loss (binary crossentropy)
    reconstruction_loss = binary_crossentropy(K.flatten(true), K.flatten(pred)) * img_width * img_height

    # KL divergence loss
    kl_loss = 1 + sigma - K.square(mu) - K.exp(sigma)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    # Total loss = 50% rec + 50% KL divergence loss
    return K.mean(reconstruction_loss + kl_loss)

Такую сложную структуру модели подсмотрел в данной работе - https://arxiv.org/pdf/1511.06434.pdf.
(В работе указана картинка, не сам код) Да, она выглядит громоздкой, но эта цена за более качественную генерацию изображений.

In [ ]:

i = layers.Input(shape=IMG_SHAPE)
cx = layers.Conv2D(filters=128, kernel_size=5, strides=2, padding='same', activation='relu')(i)
cx = layers.BatchNormalization()(cx)
cx = layers.Conv2D(filters=256, kernel_size=5, strides=2, padding='same', activation='relu')(cx)
cx = layers.BatchNormalization()(cx)
cx = layers.Conv2D(filters=512, kernel_size=5, strides=2, padding='same', activation='relu')(cx)
cx = layers.BatchNormalization()(cx)
cx = layers.Conv2D(filters=1024, kernel_size=5, strides=2, padding='same', activation='relu')(cx)
cx = layers.BatchNormalization()(cx)
x  = layers.Flatten()(cx)
x  = layers.Dense(20, activation='relu')(x)
x  = layers.BatchNormalization()(x)

mu = layers.Dense(latent_dim)(x)
sigma = layers.Dense(latent_dim)(x)

latent_space = layers.Lambda(compute_latent, output_shape=(latent_dim,))([mu, sigma])

conv_shape = K.int_shape(cx)

d_i   = layers.Input(shape=(latent_dim, ))
x     = layers.Dense(conv_shape[1] * conv_shape[2] * conv_shape[3], activation='relu')(d_i)
x     = layers.BatchNormalization()(x)
x     = layers.Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
cx    = layers.Conv2DTranspose(filters=1024, kernel_size=5, strides=2, padding='same', activation='relu')(x)
cx    = layers.BatchNormalization()(cx)
cx    = layers.Conv2DTranspose(filters=512, kernel_size=5, strides=2, padding='same', activation='relu')(cx)
cx    = layers.BatchNormalization()(cx)
cx    = layers.Conv2DTranspose(filters=256, kernel_size=5, strides=2, padding='same', activation='relu')(cx)
cx    = layers.BatchNormalization()(cx)
cx    = layers.Conv2DTranspose(filters=128, kernel_size=5, strides=2, padding='same', activation='relu')(cx)
cx    = layers.BatchNormalization()(cx)
o     = layers.Conv2DTranspose(filters=3, kernel_size=3, activation='sigmoid', padding='same')(cx)

encoder = Model(i, latent_space)
decoder = Model(d_i, o)
vae_outputs = decoder(encoder(i))
vae = Model(i,vae_outputs)

In [ ]:
encoder.summary()

In [ ]:
decoder.summary()

In [ ]:
vae.compile(optimizer='adam', loss=kl_reconstruction_loss)
history = vae.fit(train_gen, epochs=40)

А тут я изобразил сетку сгенерированных объектов из декодера, при передаче ему точек (x,y)

In [ ]:
generator_model = decoder
x_values = np.linspace(-5, 5, 30)
y_values = np.linspace(-5, 5, 30)
figure = np.zeros((112 * 30, 112 * 30,3))
for ix, x in enumerate(x_values):
    for iy, y in enumerate(y_values):
        latent_point = np.array([[x, y]])
        generated_image = generator_model.predict(latent_point)[0]
        figure[ix*112:(ix+1)*112, iy*112:(iy+1)*112] = generated_image
plt.figure(figsize=(15, 15))
plt.imshow(figure, extent=[5,-5,5,-5])
plt.show()

Ниже визуализированы кластеры классов объектов из latent space

In [ ]:
def plot_label_clusters(encoder, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean = encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c = labels.classes)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()

In [ ]:
test_images = glob.glob('train_mini/**/*.jpg', recursive= True)
X_train = np.zeros((len(test_images), img_width, img_height, 3))
for i in range(len(test_images)):
  image = tf.keras.preprocessing.image.load_img(test_images[i], target_size= (img_width, img_height))
  X_train[i] = img_to_array(image) / 255

In [ ]:
plot_label_clusters(encoder, X_train, train_gen)

А тут  создаются изображения для гифки, гифка показывает переход из одного класса в другой

In [ ]:
!mkdir images_for_gif

In [ ]:
!cd images_for_gif
for x in np.arange(0,5,0.1):
    latent_point = np.array([[x, -1]])
    image = generator_model.predict(latent_point)[0]
    plt.imshow(image)
    plt.savefig('images_for_gif/' +'image' + str(x) + '.png')

In [ ]:
anim_file = 'cvae.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('images_for_gif/image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
embed.embed_file(anim_file)